In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
from find_image import FindImage
import os
from dotenv import load_dotenv
import json 

2023-09-12 22:35:06.827991: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-12 22:35:06.859449: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-12 22:35:06.859823: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-12 22:35:07.575848: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
load_dotenv()

True

In [3]:
s3 = FindImage(True, bucket_name=os.getenv("BUCKET_S3"), resource_name="s3")

In [4]:
# Ajustar o caminho para que o jupyter tenha permissão de escrita na pagina
s3.download_images_training(os.getenv("IMGS_TRAINING"), "pets_test")

Arquivo baixado: /home/patrick/SPTECH/TCC/grupo11-4cco-tcc/src/inteligencia_artificial/imgs_training/s/cat1/cat1-2.jpeg
Arquivo baixado: /home/patrick/SPTECH/TCC/grupo11-4cco-tcc/src/inteligencia_artificial/imgs_training/s/cat1/cat1.jpeg
Arquivo baixado: /home/patrick/SPTECH/TCC/grupo11-4cco-tcc/src/inteligencia_artificial/imgs_training/s/cat2/1abe9aa7-4609-4da5-8581-1d536383cc4b.jpg
Arquivo baixado: /home/patrick/SPTECH/TCC/grupo11-4cco-tcc/src/inteligencia_artificial/imgs_training/s/cat2/208f056f-914d-497d-9c20-8afc6675a495.jpg
Arquivo baixado: /home/patrick/SPTECH/TCC/grupo11-4cco-tcc/src/inteligencia_artificial/imgs_training/s/cat2/8914bcef-615f-4c0c-9b12-a9b4d04d72b3.jpg
Arquivo baixado: /home/patrick/SPTECH/TCC/grupo11-4cco-tcc/src/inteligencia_artificial/imgs_training/s/cat2/97f8dd51-4157-4125-a42e-63a9e310880d.jpg
Arquivo baixado: /home/patrick/SPTECH/TCC/grupo11-4cco-tcc/src/inteligencia_artificial/imgs_training/s/cat2/fiona1-2.jpeg
Arquivo baixado: /home/patrick/SPTECH/TCC/gr

In [5]:
train_dir = '/home/patrick/SPTECH/TCC/grupo11-4cco-tcc/src/inteligencia_artificial/imgs_training/s'

In [6]:
train_datagen = ImageDataGenerator(
    rescale=1.0/255,  # Normalização das intensidades dos pixels
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [7]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),  # Tamanho das imagens (ajuste conforme necessário)
    batch_size=32, # Em quantos pacotes/partes eu divido meu dataset 
    class_mode='categorical'  # Modo categórico para múltiplas classes
)

Found 19 images belonging to 6 classes.


In [8]:
base_model = tf.keras.applications.MobileNetV2(input_shape=(224, 224, 3), include_top=False, weights='imagenet')
base_model.trainable = False

model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(train_generator.num_classes, activation='softmax')
])

2023-09-12 22:35:24.108254: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-09-12 22:35:24.108603: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [9]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [10]:
model.fit(train_generator, epochs=9)

Epoch 1/9
1/1 [==============================] - 2s 2s/step - loss: 2.4961 - accuracy: 0.1053
Epoch 2/9
1/1 [==============================] - 1s 550ms/step - loss: 1.4275 - accuracy: 0.4211
Epoch 3/9
1/1 [==============================] - 1s 524ms/step - loss: 1.1769 - accuracy: 0.5263
Epoch 4/9
1/1 [==============================] - 1s 527ms/step - loss: 0.5915 - accuracy: 0.7368
Epoch 5/9
1/1 [==============================] - 1s 558ms/step - loss: 0.7122 - accuracy: 0.7895
Epoch 6/9
1/1 [==============================] - 1s 532ms/step - loss: 0.3899 - accuracy: 0.8947
Epoch 7/9
1/1 [==============================] - 1s 552ms/step - loss: 0.3439 - accuracy: 0.9474
Epoch 8/9
1/1 [==============================] - 1s 555ms/step - loss: 0.2373 - accuracy: 0.9474
Epoch 9/9
1/1 [==============================] - 1s 533ms/step - loss: 0.1841 - accuracy: 1.0000


In [11]:
class_labels = list(train_generator.class_indices.keys())
print(class_labels)

['cat1', 'cat2', 'cat3', 'cat4', 'dog1', 'dog2']


In [18]:
dict_class_labels = {}
for index, class_label in enumerate(class_labels):
  dict_class_labels[index] = class_label

print(dict_class_labels)

{0: 'cat1', 1: 'cat2', 2: 'cat3', 3: 'cat4', 4: 'dog1', 5: 'dog2'}


In [16]:
class_names_file = 'class_names.json'
with open(class_names_file, 'w') as file:
    json.dump(dict_class_labels, file)

In [13]:
model.save(os.getenv("MODEL_PATH"))

INFO:tensorflow:Assets written to: /home/patrick/SPTECH/TCC/grupo11-4cco-tcc/src/inteligencia_artificial/modelv1/assets


INFO:tensorflow:Assets written to: /home/patrick/SPTECH/TCC/grupo11-4cco-tcc/src/inteligencia_artificial/modelv1/assets
